# Gravity column

This PRST example is based on the "1ph/gravityColumn.m" example found in MRST.

## MATLAB code

The code we are trying to convert to Python is the following:

```
%% My First Flow Solver: Gravity Column
% In this example, we introduce a simple pressure solver and use it to
% solve the single-phase pressure equation
%
% $$\nabla\cdot v = q, \qquad
%    v=\textbf{--}\frac{K}{\mu} \bigl[\nabla p+\rho g\nabla z\bigr],$$
%
% within the domain [0,1]x[0,1]x[0,30] using a Cartesian grid with
% homogeneous isotropic permeability of 100 mD. The fluid has density 1000
% kg/m^3 and viscosity 1 cP and the pressure is 100 bar at the top of the
% structure.
%
% The purpose of the example is to show the basic steps for setting up,
% solving, and visualizing a flow problem. More details on the grid
% structure, the structure used to hold the solutions, and so on, are given
% in the <simpleBC.html basic flow-solver tutorial>.
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end

%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
bc  = pside([], G, 'TOP', 100.*barsa());

%% Assemble and solve the linear system
% To solve the flow problem, we use the standard two-point
% flux-approximation method (TPFA), which for a Cartesian grid is the same
% as a classical seven-point finite-difference scheme for Poisson's
% equation. This is done in two steps: first we compute the
% transmissibilities and then we assemble and solve the corresponding
% discrete system.
T   = computeTrans(G, rock);
sol = incompTPFA(initResSol(G, 0.0), G, T, fluid, 'bc', bc);

%% Plot the face pressures
clf
plotFaces(G, 1:G.faces.num, convertTo(sol.facePressure, barsa()));
set(gca, 'ZDir', 'reverse'), title('Pressure [bar]')
view(3), colorbar
set(gca,'DataAspect',[1 1 10]);
%%
displayEndOfDemoMessage(mfilename)
```

The rest of the example will walk through this example line for line, displaying the PRST examples.

## Setup

**Start by importing PRST.**

In [18]:
# For Python 3 compatibility
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np

import prst
import prst.incomp as incomp
import prst.gridprocessing as gridprocessing
import prst.utils as utils
import prst.params as params
import prst.solvers as solvers

In [2]:
help(prst)

Help on package prst:

NAME
    prst - Python Reservoir Simulation Toolbox.

FILE
    m:\prst\prst\__init__.py

PACKAGE CONTENTS
    gridprocessing
    incomp (package)
    io
    params (package)
    plotting
    solvers
    utils (package)

DATA
    __all__ = ['gridprocessing', 'io', 'incomp', 'plotting', 'utils', 'par...




## Import the "incomp" module
```
try
    require incomp
catch %#ok<CTCH>
    mrstModule add incomp
end
```

In [3]:
import prst.incomp

```
%% Define the model
% To set up a model, we need: a grid, rock properties (permeability), a
% fluid object with density and viscosity, and boundary conditions.
gravity reset on
```

In [4]:
# PRST uses a single module-level variable to set the gravity
params.gravity_reset()
print("Gravity vector:", params.gravity)

Gravity vector: [ 0.       0.       9.80665]


## Build the grid, setup fluid

```G          = cartGrid([1, 1, 30], [1, 1, 30]);
G          = computeGeometry(G);
rock.perm  = repmat(0.1*darcy(), [G.cells.num, 1]);
fluid      = initSingleFluid('mu' ,    1*centi*poise, ...
                             'rho', 1014*kilogram/meter^3);
                             ```

In [5]:
G = gridprocessing.cartGrid([1, 1, 30], [1, 1, 30])
print(G)

<PRST grid
  gridType: [u'tensorGrid', u'cartGrid']
  cells: ['facePos', 'num', 'indexMap', 'faces']
  cartDims: [ 1  1 30]
  gridDim: 3
  faces: ['neighbors', 'nodes', 'num', 'tag', 'nodePos']
  nodes: ['num', 'coords']
>


**Computing the geometry modifies the original grid in place, no assignment is 
necessary. Note that specifying the module is necessary in PRST.
This avoids polluting the global namespace, at the cost of more verbosity.**

In [6]:
gridprocessing.computeGeometry(G)

INFO:prst:Computing normals, areas and centroids...
INFO:prst:Computing cell volumes and centroids


In [7]:
print(G)

<PRST grid
  gridType: [u'tensorGrid', u'cartGrid', u'computeGeometry']
  cells: ['facePos', 'centroids', 'num', 'volumes', 'faces', 'indexMap']
  cartDims: [ 1  1 30]
  gridDim: 3
  faces: ['neighbors', 'nodePos', 'tag', 'centroids', 'num', 'normals', 'nodes', 'areas']
  nodes: ['num', 'coords']
>


In [8]:
from prst.params.rock import Rock
from prst.utils.units import *
rock = Rock(G, perm=0.1*darcy, poro=1)

In [9]:
from prst.incomp.fluid import SingleFluid
fluid = SingleFluid(viscosity=1*centi*poise, density=1014*kilogram/meter**3)
print(fluid)

{'mu': 0.001, 'rho': 1014.0}


**Set up boundary conditions. Constant pressure at top face.**

In [10]:
#bc  = pside([], G, 'TOP', 100.*barsa());
from prst.params.wells_and_bc import BoundaryCondition, _boundaryCellsSubset
try:
    reload(prst.params.wells_and_bc)
except Exception as e:
    print("couldnt reload", e)

bc = BoundaryCondition()
bc.addPressureSide(G, "top", 100*bar)



In [11]:
print(bc)

{'sat': None, 'type': array([[u'pressure']], 
      dtype='<U8'), 'value': array([[ 10000000.]]), 'face': array([[120]])}


## Assemble and solve the linear system
```
%% Assemble and solve the linear system
% To solve the flow problem, we use the standard two-point
% flux-approximation method (TPFA), which for a Cartesian grid is the same
% as a classical seven-point finite-difference scheme for Poisson's
% equation. This is done in two steps: first we compute the
% transmissibilities and then we assemble and solve the corresponding
% discrete system.
T   = computeTrans(G, rock);
sol = incompTPFA(initResSol(G, 0.0), G, T, fluid, 'bc', bc);
```

In [14]:
T = computeTrans(G, rock)
resSol = initResSol(G, p0=0.0)

NameError: name 'initResSol' is not defined

In [ ]:
# prst/solvers.py

def capPressureRHS():
    pass

def computePressureRHS():
    pass

def incompTPFA():
    pass

def _dynamic_quantities():
    pass

def _compute_trans():
    pass

## incompTPFA

In [ ]:
state = resSol
G = G
T = T
fluid = fluid
wells = None
bc = bc
src = None
LinSolve = None
MatrixOutput = False
verbose = None
condition_number = False
gravity = None

In [ ]:
print(bc)

In [26]:
import importlib
help(importlib.import_module
)

Help on function import_module in module importlib:

import_module(name, package=None)
    Import a module.
    
    The 'package' argument is required when performing a relative import. It
    specifies the package to use as the anchor point from which to resolve the
    relative import to an absolute import.



In [8]:
import importlib
incomp = importlib.import_module("prst.incomp")
incomp.utils = importlib.import_module("..utils", incomp)

ValueError: 'package' not set to a string

In [5]:
incomp.utils

AttributeError: 'module' object has no attribute 'utils'